Arthur Catarino de Oliveira  
Guilherme Fidélis Freire  
Mateus Poddis Correa

Neste exercicio criamos um subgrupo de aviões chamado AC que são os aviões que so podem ser pilotados por "pilotos coringas", no caso o MD-11, pilotos coringas sao pilotos que podem pilotar qualquer avião.

##Dados
- A: Conjunto dos aviões.
- AC: Conjunto dos avioes que so podem ser pilotados por pilotos coringas.
- PC: Pilotos coringas que podem pilotar qualquer avião.
- Aci: Custo de compra do avião i, i E A.
- Ali: Lucro gerado pelo avião i, i E A.
- Api: Pilotos disponiveis para o avião i, i E (A - AC).
- Ami: Custo de manutenção do avião i, i E A.
- Vmax: Verba máxima disponivel para a compra de aviões.
- Mmax: Capacidade maxima de manutenção maxima da oficina da empresa.

##Variaveis de decisão
- Xi, i E A, quantidade de aviões que serao comprados.
- Yi, i E A, quantidade de pilotos coringas alocados para cada avião.

##Função Objetivo:
$$
Maximizar: \sum_{i E A}^{} X_i*Al_i
$$

##Restrições:
1) Restrição que respeita a verba máxima da empresa, impedindo que seja gasto mais que o disponivel na compra dos aviões.

 $\sum_{i E A}^{} X_i*Ac_i$ <= Vmax  


2) Restrição que respeita a quantidade maxima de aviões que a ofinica da empresa teria capacidade de cuidar.

 $\sum_{i EA}^{}X_i*Am_i$ <= Mmax

 3) Restrição que respeita a quantidade maxima de pilotos coringas.

 $\sum_{i EA}^{}Y_i$ <= PC

 4) Restrição que garante que havera pilotos para os aviões comprados (Para aviões que tem nao sao pilotados somente por pilotos coringas).

$X_i <= \frac{Ap_i+Y_i}{2}$  $∀ i E (A-AC)$

Para os aviões que so podem ser pilotados por pilotos coringas.

$X_i <= \frac{Y_i}{2}$  $∀ i E AC$  

5) $X_i, Y_i$ >= 0 nao negatividade.

In [ ]:
!pip install pulp
import pulp

Organizamos os dados com um dicionario (nomeDoAviao,Objeto Aviao).

In [ ]:
class Aviao:
  def __init__(self,custo,lucro,manutencao,pilotos,coringa):
    self.custo = custo
    self.lucro = lucro
    self.manutencao = manutencao
    self.pilotos = pilotos
    self.coringa = coringa

avioes = {
  "BOEING 717":Aviao(5.1,330,1,30,False),
  "BOEING 737-500":Aviao(3.6,300,0.75,20,False),
  "MD-11":Aviao(6.8,420,1.67,0,True)
}

verbaMaxima = 220
pilotosCoringa = 10
CapacidadeManutencao = 40

Inicializando o modelo

In [ ]:
modelo = pulp.LpProblem("EmpresaDeViaçãoAéreaBrasileira",pulp.LpMaximize)


x = pulp.LpVariable.dicts("X",list(avioes.keys()),lowBound=0)
y = pulp.LpVariable.dicts("Y",list(avioes.keys()),lowBound=0)

Função objetivo : $$
Maximizar: \sum_{i E A}^{} X_i*Al_i
$$

In [ ]:
modelo += pulp.lpSum([x[i]*avioes[i].lucro  for i in x.keys()])

Restrições:

1 - $\sum_{i E A}^{} X_i*Ac_i$ <= Vmax  (Garante que o limite de verba seja respeitado).

In [ ]:
modelo += pulp.lpSum([x[i]*avioes[i].custo for i in x.keys()]) <= verbaMaxima

 2 - $\sum_{i EA}^{}X_i*Am_i$ <= Mmax (Garante que o limite de reparos da oficina seja respeitado).

In [ ]:
modelo += pulp.lpSum([x[i]*avioes[i].manutencao for i in x.keys()]) <= CapacidadeManutencao

3 -  $\sum_{i EA}^{}Y_i$ <= PC (Garante que o limite de pilotos coringas seja respeitado).

In [ ]:
modelo+= pulp.lpSum(y[i] for i in y.keys()) <= pilotosCoringa

4 a) - $X_i <= \frac{Ap_i+Y_i}{2}$  $∀ i E (A-AC)$ (Limita a compra de aviões ao numero de pilotos disponiveis para aquele avião)

b) - $X_i <= \frac{Y_i}{2}$  $∀ i E AC$ (Versão dos aviões que so podem ser pilotados por pilotos coringas)

In [ ]:
for a in avioes.keys():
  if(not avioes[a].coringa):
    modelo += x[a] <= (avioes[a].pilotos+y[a])/2
  else:
    modelo += x[a] <= y[a]/2

Imprimi o modelo


In [ ]:
print(modelo)

EmpresaDeViaçãoAéreaBrasileira:
MAXIMIZE
330*X_BOEING_717 + 300*X_BOEING_737_500 + 420*X_MD_11 + 0.0
SUBJECT TO
_C1: X_BOEING_717 + 0.75 X_BOEING_737_500 + 1.67 X_MD_11 <= 40

_C2: 5.1 X_BOEING_717 + 3.6 X_BOEING_737_500 + 6.8 X_MD_11 <= 220

_C3: X_BOEING_717 + 0.75 X_BOEING_737_500 + 1.67 X_MD_11 <= 40

_C4: Y_BOEING_717 + Y_BOEING_737_500 + Y_MD_11 <= 10

_C5: X_BOEING_717 - 0.5 Y_BOEING_717 <= 15

_C6: X_BOEING_737_500 - 0.5 Y_BOEING_737_500 <= 10

_C7: X_MD_11 - 0.5 Y_MD_11 <= 0

VARIABLES
X_BOEING_717 Continuous
X_BOEING_737_500 Continuous
X_MD_11 Continuous
Y_BOEING_717 Continuous
Y_BOEING_737_500 Continuous
Y_MD_11 Continuous



Solução do problema

In [ ]:
status = modelo.solve()
print('Status: ', pulp.LpStatus[status])
print('Funcao objetivo: ', round(modelo.objective.value(),2))
print('Soluçoes')
for i in x.keys():
  print("X-" + i,": ",x[i].value())

for i in x.keys():
  print("Y-" + i,": ",y[i].value())

Status:  Optimal
Funcao objetivo:  10050.0
Soluçoes
X-BOEING 717 :  15.0
X-BOEING 737-500 :  10.0
X-MD-11 :  5.0
Y-BOEING 717 :  0.0
Y-BOEING 737-500 :  0.0
Y-MD-11 :  10.0
